## Configuration

In [99]:
### Configuration
# Import dependencies
import os
from collections import Counter
import summerplaylist

# Environment variables
CLIENTID = os.environ.get('CLIENTID')
CLIENTSECRET = os.environ.get('CLIENTSECRET')
REDIRECT_URI = 'https://example.com/callback'

## Log In

In [ ]:
# Authenticate to Spotify
spotify, userID, user_token = authenticateToSpotify(CLIENTID, CLIENTSECRET, REDIRECT_URI)

## Functions

In [74]:
# def getPlaylistID(playlistName):
#     with spotify.token_as(user_token):
#         # Check if playlist already exists; if not create a new one
#         userPlaylists = pd.DataFrame([{'id':x.id, 'name':x.name} for x in spotify.playlists(userID, limit=40).items])
#         if not (userPlaylists['name'].eq(playlistName)).any():
#             # Create a playlist for the user
#             newPlaylist = spotify.playlist_create(userID, playlistName, public=False, description='Recent albums from new artists')
#             playlistID = newPlaylist.id
#         # Else just get playlistID
#         else:
#             playlistID = userPlaylists.loc[userPlaylists['name']==playlistName, 'id'][0]
#     return playlistID

In [15]:
# def fetchArtistsFromPlaylist(playlistID):
#     # Get all artists in the list from spotify
#     nestedList = [[y.id for y in x.track.artists] for x in spotify.playlist_items(playlistID).items]
#     # Unnest the list 
#     artists = [item for sublist in nestedList for item in sublist]
#     # Deduplicate by converting to set
#     artists = list(set(artists))
#     return artists

In [87]:
# def fetchRecentAlbums(artistIDs):
#     # Create empty list
#     albums = []
#     # Loop through list of artists
#     for artistID in artistIDs:
#         try:
#             # Query the spotify API for that artist's albums
#             query = spotify.artist_albums(artistID, limit=50, include_groups=['album']).items
#             result = pd.DataFrame([{'albumID':x.id, 'albumReleaseDate':x.release_date} for x in query])
#             # Filter out albums older than three years
#             result['albumReleaseDate'] = pd.to_datetime(result['albumReleaseDate'])
#             result = result.loc[(result["albumReleaseDate"]>=MINDATE) & (result["albumReleaseDate"]<=MAXDATE)]
#             # Get only the list of albumURIs
#             additions = list(result['albumID'])
#         except: 
#             # If exception, fill in blank data
#             additions = []
#         finally:
#             # Wait 2 seconds so we do not exceed our API call allowance
#             time.sleep(2)
#             # Append the recent albums to the list
#             albums += additions
#     # Remove duplicates
#     albums = list(set(albums))
#     return albums


In [77]:
# def addToPlaylist(URIList, playlistID):
#     with spotify.token_as(user_token):
#         # Counter
#         n = 0
#         # Add songs in chunks of 100
#         while n < len(URIList):
#             toAdd = URIList[n:(n+100)]
#             spotify.playlist_add(playlistID, toAdd, position=None)
#             # Wait two seconds to avoid exceeding API limits
#             time.sleep(2)
#             # Increment counter
#             n += 100

## Collect artists from playlist

In [82]:
# Read in favorite new artists from culled ArtistHasPotential playlist
winnersPlaylist = getPlaylistID('New Artists I Liked')
# Get all the artistIDs in that playlist
artistIDs = fetchArtistsFromPlaylist(winnersPlaylist)
# Sort and deduplicate artists by value counts (get value count dict; sort this dict)
artistIDs = list(dict(sorted(Counter(artistIDs).items(), key=lambda x:x[1], reverse=True)).keys())
artistIDs

['1GmsPCcpKgF9OhlNXjOsbS', '0QWrMNukfcVOmgEU0FEDyD', '5jQsMNuCW0iZeFTz6YUK8K']

## Get the recent albums for each artist

In [88]:
# Find recent songs < 3 years old from new artists
recentAlbums = fetchRecentAlbums(artistIDs, maxAge=3)
recentAlbums

['33cj3kzLqVOg9zvy69Wrc8',
 '63ButSDtUefzCmdE3AXdor',
 '1mu38xw1WDRabnfwlvGqNO',
 '7H7nlUOKxq1u3Ev2TP5ib5',
 '5u8SP8bbxcA7zJoAusBfgc',
 '0G9S31E4tXEPAdLTR6Os3M',
 '2OAeKCwp9IyZou5eFsiSa0',
 '68L5xVV9wydotfDXEik7eD']

## Get all the tracks on these playlists

In [93]:
def fetchTracks(albumIDs):
    with spotify.token_as(user_token):
        # Cut the df into chunks
        n = 0
        trackURIs = []
        while n < len(albumIDs):
            albumsToGet = albumIDs[n:(n+20)]
            try:
                    # Query spotify for that album's tracks
                    nestedList = [[y.uri for y in x.tracks.items] for x in spotify.albums(albumsToGet, market='from_token')]
                    # Unnest the list 
                    tracks = [item for sublist in nestedList for item in sublist]
                    trackURIs += tracks
                    # Wait 2 seconds so we do not exceed our API call allowance
                    time.sleep(2)
            except:
                    # If there is an error in the batch, go through each separately
                    for albumID in albumsToGet:
                        try:
                            nestedList = [[y.uri for y in x.tracks.items] for x in spotify.album(albumID, market='from_token')]
                            # Unnest the list 
                            tracks = [item for sublist in nestedList for item in sublist]
                            trackURIs += tracks
                            # Wait 2 seconds so we do not exceed our API call allowance
                            time.sleep(2)
                        except: 
                            pass
            finally:
                n += 20
    # Drop duplicates
    trackURIs = trackURIs.unique()
    return trackURIs

In [94]:
recentTracks = fetchTracks(recentAlbums)
recentTracks

AttributeError: 'list' object has no attribute 'unique'

## Add recent albums to a playlist

In [92]:
# Warning: This will make a new playlist
playlistID = getPlaylistID('New Artists to Explore Further')
# Add songs to the playlist
addToPlaylist(recentTracks, playlistID)

In [ ]:
# Cull both newFaves and oldFaves into CurrentYearSongCandidates